In [1]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from skimage.feature import hog
from skimage.color import rgb2grey
import os
import numpy as np
from sklearn import metrics
from sklearn.utils import shuffle
import skvideo.io

/home/gbt/Github/Violence Detection/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [27]:
def get_movie_list_from_folder(folder):
    movies = []
    for file in os.listdir(folder):
        movie = os.path.join(folder, file)
        movies.append(movie)
    return movies

In [28]:
violence_path = 'files/movies/Violence/'
nonviolence_path = 'files/movies/NonViolence/'

violence_movies = get_movie_list_from_folder(violence_path)
nonviolence_movies = get_movie_list_from_folder(nonviolence_path)

In [29]:
def compute_hogs(file):
    hogs = []
    videogen = skvideo.io.vreader(file)
    for frame in videogen:
        greyscale = rgb2grey(frame)
        frame_hog = hog(greyscale, visualise=False, feature_vector=True)
        hogs.append(frame_hog)    
    hogs = np.array(hogs)
    return hogs

def save_hogs_for_movie_list(movies, save_folder):
    for i, x in enumerate(movies):
        print(i)
        try:
            if not os.path.isfile(save_folder + os.path.basename(x) + '.npy'):
                hogs = compute_hogs(x)
                np.save(save_folder + os.path.basename(x), hogs)
            else:
                print(save_folder + os.path.basename(x), 'exists')
        except Exception as e:
            print(e)

In [31]:
save_hogs_for_movie_list(violence_movies, 'hogs/violence/')
save_hogs_for_movie_list(nonviolence_movies, 'hogs/nonviolence/')

0
1
2
3
4
5
6
7
8
9
10
11


KeyboardInterrupt: 

In [2]:
violence_hogs = [np.load('hogs/violence/' + file) for file in os.listdir('hogs/violence')]
violence_labels = [1 for i in range(len(violence_hogs))]

nonviolence_hogs = [np.load('hogs/nonviolence/' + file) for file in os.listdir('hogs/nonviolence')]
nonviolence_labels = [0 for i in range(len(nonviolence_hogs))]

all_hogs = violence_hogs + nonviolence_hogs
all_labels = violence_labels + nonviolence_labels

In [3]:
def average_hogs():
    return [np.mean(hogs, axis=0) for hogs in all_hogs]

In [4]:
def sum_hogs():
    return [np.sum(hogs, axis=0) for hogs in all_hogs]

In [5]:
features = sum_hogs()
features = np.array(features)
print(features.shape)

(94, 86184)


In [12]:
features, all_labels = shuffle(features, all_labels)
estimator = LinearSVC()
scores = cross_val_score(estimator, features, all_labels, cv=10)
print('mean', np.mean(scores))
print('std', np.std(scores))

mean 0.894242424242
std 0.0672184845169
